In [18]:
import google.generativeai as genai
from serpapi import GoogleSearch
import requests
from readability import Document

# SerpAPI Setup
api_key = "c9691012c0d88d2800f8b26247609a0231402853f0c2cb6c89c3d9d1fa39a41f" # Replace with your SerpAPI key
article_citation_id = "1NtVbf1efHoJ"

urls = []

start = 0  # Pagination index
MAX_RESULTS = 100

while len(urls) < MAX_RESULTS:
    params = {
        "engine": "google_scholar",
        "cites": article_citation_id,
        "api_key": api_key,
        "start": start
    }

    search = GoogleSearch(params)
    results = search.get_dict()
    organic_results = results.get("organic_results", [])

    if not organic_results:
        break  # No more results, exit the loop

    for result in organic_results:
        urls.append(result["link"])
        if len(urls) >= MAX_RESULTS:
            break  # Stop once we reach the desired number of articles

    start += 10  # Move to the next page

# Save the URLs to a file
with open("urls.txt", "w") as file:
    for url in urls:
        file.write(url + "\n")

print(f"Retrieved {len(urls)} URLs (limited to {MAX_RESULTS}).")

Retrieved 100 URLs (limited to 100).


In [6]:
# Function to fetch and parse article content
def fetch_article_content(url):
    try:
        response = requests.get(url, timeout=10)
        response.raise_for_status()  # Raise an error for bad status codes
        doc = Document(response.text)
        return doc.summary(), doc.title()
    except Exception as e:
        print(f"Failed to process {url}: {e}")
        return None, None

# Loop through URLs and fetch content
articles = []
for url in urls:
    content, title = fetch_article_content(url)
    if content:
        articles.append({"url": url, "title": title, "content": content})
        print(f"Fetched: {title}")

print(f"Successfully fetched content for {len(articles)} articles.")

Failed to process https://www.cell.com/cell-systems/fulltext/S2405-4712(19)30234-0: 403 Client Error: Forbidden for url: https://www.cell.com/cell-systems/fulltext/S2405-4712(19)30234-0
Failed to process https://ascopubs.org/doi/abs/10.1200/CCI.18.00069: 403 Client Error: Forbidden for url: https://ascopubs.org/doi/abs/10.1200/CCI.18.00069
Failed to process https://www.cell.com/trends/cell-biology/fulltext/S0962-8924(22)00240-9: 403 Client Error: Forbidden for url: https://www.cell.com/trends/cell-biology/fulltext/S0962-8924(22)00240-9
Fetched: Radware Bot Manager Captcha
Failed to process https://www.tandfonline.com/doi/abs/10.1080/19336918.2022.2055520: 403 Client Error: Forbidden for url: https://www.tandfonline.com/doi/abs/10.1080/19336918.2022.2055520
Fetched: Multiscale Agent-Based and Hybrid Modeling of the Tumor Immune Microenvironment
Failed to process https://www.cell.com/trends/cancer/fulltext/S2405-8033(20)30017-0: 403 Client Error: Forbidden for url: https://www.cell.com/t

error getting summary: 
Traceback (most recent call last):
  File "/Users/aaronrusk/Desktop/Y390/Y390project/env/lib/python3.11/site-packages/readability/readability.py", line 213, in summary
    self._html(True)
  File "/Users/aaronrusk/Desktop/Y390/Y390project/env/lib/python3.11/site-packages/readability/readability.py", line 148, in _html
    self.html = self._parse(self.input)
                ^^^^^^^^^^^^^^^^^^^^^^^
  File "/Users/aaronrusk/Desktop/Y390/Y390project/env/lib/python3.11/site-packages/readability/readability.py", line 157, in _parse
    doc, self.encoding = build_doc(input)
                         ^^^^^^^^^^^^^^^^
  File "/Users/aaronrusk/Desktop/Y390/Y390project/env/lib/python3.11/site-packages/readability/htmls.py", line 21, in build_doc
    doc = lxml.html.document_fromstring(
          ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/Users/aaronrusk/Desktop/Y390/Y390project/env/lib/python3.11/site-packages/lxml/html/__init__.py", line 738, in document_fromstring
    raise 

Failed to process https://dash.harvard.edu/handle/1/37370548: Document is empty
Fetched: [no-title]
Fetched: Modelling the effects of temporal variations of blood flow in tumours - ORA - Oxford University Research Archive
Failed to process https://www.researchgate.net/profile/Renji-Han/publication/360754695_In_Silico_Evaluation_of_Paxlovid's_Pharmacometrics_for_SARS-CoV-2_A_Multiscale_Approach/links/628840403303d263c46bea57/In-Silico-Evaluation-of-Paxlovids-Pharmacometrics-for-SARS-CoV-2-A-Multiscale-Approach.pdf: 403 Client Error: Forbidden for url: https://www.researchgate.net/profile/Renji-Han/publication/360754695_In_Silico_Evaluation_of_Paxlovid's_Pharmacometrics_for_SARS-CoV-2_A_Multiscale_Approach/links/628840403303d263c46bea57/In-Silico-Evaluation-of-Paxlovids-Pharmacometrics-for-SARS-CoV-2-A-Multiscale-Approach.pdf
Fetched: Paralelización de Modelos de Simulación de Crecimiento Tumoral
Fetched: [no-title]
Fetched: Research Portal
Fetched: Computational Modelling of Cancer Syst

In [15]:
sample_article_content = articles[145]["content"]
print(sample_article_content)

<html><body><div><div class="section" id="sec-2"><h2 class="">Introduction</h2><p id="p-8">Traumatic brain injury (TBI) is a severe condition affecting an increasing number of people, estimated from 27 million to 69 million each year [<a id="xref-ref-1-1" class="xref-bibr" href="#ref-1">1</a>, <a id="xref-ref-2-1" class="xref-bibr" href="#ref-2">2</a>]. Brain trauma often leads to serious health problems including persistent impairments in cognitive, sensory, and motor function. Unfortunately, there is no treatment yet for this complex condition and brain tissue in mammals has a very limited capacity to repair. In contrast, zebrafish larvae possess high regenerative capacities and can fully repair brain injuries within a few days [<a id="xref-ref-3-1" class="xref-bibr" href="#ref-3">3</a>]. They are optically transparent which allows for real-time <em>in vivo</em> visualisation of tissular and cellular processes. Moreover, these fish share a high degree of similarity in genetics, neuro

In [16]:
def analyze_article_with_gemini(content):
    try:
        model = genai.GenerativeModel("gemini-pro")
        prompt = (
            "Summarize the key conclusions about the use of PhysiCell "
            "in the following academic article:\n\n" + content
        )
        response = model.generate_content(prompt)
        return response.text
    except Exception as e:
        print(f"Failed to analyze content: {e}")
        return None


    
article_content = articles[145]["content"]  # Replace with actual content
analysis_result = analyze_article_with_gemini(article_content)
print("Article Analysis:")
print(analysis_result)

Article Analysis:
- Microgilia are cells that play an important role in the brain's response to injury. 
- Previous research has shown that microglia are recruited to the site of injury and that they help to clear away damaged tissue. 
- In zebrafish larvae, researchers have found that microglia play a role in wound closure after brain injury. 
- The researchers used a combination of live imaging, mathematical modeling, and genetic manipulations to show that microglia are necessary for wound closure and that they exert mechanical forces on the surrounding tissue to pull the wound together. 
- This study provides new insights into the role of microglia in brain repair and suggests that targeting microglia could be a potential therapeutic strategy for treating brain injuries.


In [20]:
from sklearn.feature_extraction.text import TfidfVectorizer
import pandas as pd

summaries = [article['summary'] for article in articles]

# Compute TF-IDF scores
vectorizer = TfidfVectorizer()
tfidf_matrix = vectorizer.fit_transform(summaries)
importance_scores = tfidf_matrix.sum(axis=1).A1  # Sum TF-IDF scores for each summary

# Add scores to DataFrame
for i, score in enumerate(importance_scores):
    articles[i]['importance'] = score


KeyError: 'summary'